In [56]:
 
import networkx as nx

def product_G(G1,G2):
    # add more zeros to both networks
    print ('G1 and G2')
    print (len(G1))
    print (len(G2))
    G = nx.Graph()
    aa =[(u,v) for u in G1 for v in G2]
    ee = []
    for a1 in aa:
        for a2 in aa:
            u1 = a1[0]
            u2 = a1[1]
            v1 = a2[0]
            v2 = a2[1]
            if has_a_link_2(G1,G2,u1,u2,v1,v2):
                ee.append((a1,a2))
    ####################
    G.add_edges_from(ee)
    ##############################
    return G
def has_a_link_2(G1,G2,u1,u2,v1,v2):
    b1 = G1.has_edge(u1,v1)
    b2 = G2.has_edge(u2,v2)
    b = False
    if b1 == True and b2 == True:
        b = True
    elif b1 == False and b2 == False:
        b = True
    else:
        b = False
    return b

def common_sub_structure_2(G1,G2):
    # add more zeros to both networks
    print ('G1 and G2')
    print (len(G1))
    print (len(G2))
    G = nx.Graph()
    #aa =[(u,v) for u in G1 for v in G2]
    aa =[(u,u) for u in G1 if u in G2]
    ee = []
    for a1 in aa:
        for a2 in aa:
            if a1 != a2:
                u1 = a1[0]
                u2 = a1[1]
                v1 = a2[0]
                v2 = a2[1]
                if has_a_link_2(G1,G2,u1,u2,v1,v2):
                    ee.append((a1,a2))
    ####################
    G.add_edges_from(ee)
    ##############################
    # find max clique
    ll = 0
    if len(G) > 10:
        clq2 = list(nx.find_cliques(G))
        maxcl = clq2[0]
        size = len(maxcl)
        for cl in clq2:
            if len(cl) > size:
                maxcl = cl
                size = len(cl)
        ll = 1 - len(maxcl)/110.0
    ############
    for a in maxcl:
        u1 = a[0]
        u2 = a[1]
    c1 = [a[0] for a in maxcl]
    c2 = [a[1] for a in maxcl]
    return c1,c2

def has_a_link(G1,G2,u,v):
    b1 = G1.has_edge(u,v)
    b2 = G2.has_edge(u,v)
    b = False
    if b1 == True and b2 == True:
        b = True
    elif b1 == False and b2 == False:
        b = True
    else:
        b = False
    return b

def common_sub_structure(G1,G2):
    # add more zeros to both networks
    G = nx.Graph()
    nn = [u for u in G1 if u in G2]
    aa =[(u,v) for u in nn for v in nn if u!=v and has_a_link(G1,G2,u,v)]
    G.add_edges_from(aa)
    ##############################
    clq2 = list(nx.find_cliques(G))
    b = [len(bb) for bb in clq2 if len(bb)>28]
    print (b)
    if len(clq2) > 0:
        maxcl = clq2[0]
        size = len(maxcl)
        for cl in clq2:
            if len(cl) > size:
                maxcl = cl
                size = len(cl)
    return maxcl

from itertools import product
def get_categories():
    '''
    This function generates investor category labels and assigns them an
    increasing index
    '''
    por_labels = ['Helsinki', 'Rest-Uusimaa', 'Eastern-Tavastia',
                  'South-West', 'Western-Tavastia', 'Central-Finland',
                  'South-East', 'Ostrobothnia', 'Northern-Savonia',
                  'Eastern-Finland', 'Northern-Finland']
    ia_house = ['Under-Age', 'Young', 'Middle-Age', 'Mature', 'Retired']
    ia_non_h = ['no-age']
    household = ['Households']
    non_house = ['Non-Financial', 'Financial-Insurance',
                 'General-Government', 'Non-Profit', 'Rest-World']

    houseGroups = list(product(household, por_labels, ia_house))
    nonHoGroups = list(product(non_house, por_labels, ia_non_h))
    totalGroups = nonHoGroups + houseGroups
    groups = {name: idx for idx, name in enumerate(totalGroups)}
    return groups

def update_atr(G):
    groups = get_categories()
    sector_code = {}
    postal_code = {}
    age = {}
    for node in G:
        kk = [k for k, v in groups.items() if v == int(node)]
        at = kk[0]
        sector_code.update({node:at[0]}) 
        postal_code.update({node:at[1]})
        age.update({node:at[2]}) 
    nx.set_node_attributes(G, sector_code, name='sector_code')
    nx.set_node_attributes(G, postal_code, name='postal_code')
    nx.set_node_attributes(G, age, name='age')
    return G

def update_subCommon_structure(G,clique):
    subCommon_structure = {}
    for node in G:
        inClique = 0
        if node in clique:
            inClique = 1
        subCommon_structure.update({node:inClique}) 
    nx.set_node_attributes(G, subCommon_structure, name='subCommon_structure')
    return G

def update_link_atr(G):
    d = {}
    for a,b in G.edges():
        sameA = 0
        sameR = 0
        sub_common = 0
        if G.nodes[a]['subCommon_structure'] == 1 & G.nodes[b]['subCommon_structure'] == 1:
            sub_common = 1
        if G.nodes[a]['postal_code'] == G.nodes[b]['postal_code']:
            sameR = 1
        if G.nodes[a]['age'] == G.nodes[b]['age']:
            sameA = 1
        
        d[(a,b)] = {'sameRegion': sameR, 'sameAge': sameA, 'subCommon': sub_common}

    nx.set_edge_attributes(G, d)
    return G

def attributes_for_subcommon(G):
    attrs_subcommon = {}
    for node in G:
        if G.nodes[node]['subCommon_structure'] == 1:
            attrs_subcommon.update({node:G.nodes[node]['sector_code']})
        else:
            attrs_subcommon.update({node:''})
    nx.set_node_attributes(G, attrs_subcommon, name='sector_code_subcommon')
    return G


def list_45_stocks():
    return ['FI0009013296']

outputPath = ''
stk = list_45_stocks()
#sc = stk[0]
#sc = 'FI0009000665' 
sc = 'FI0009013296'
#B = nx.read_graphml('FI0009000665'+'_before.graphml')
#D = nx.read_graphml('FI0009000665'+'_during.graphml')
#B = nx.read_graphml(outputPath+sc+'_before.graphml')
#D = nx.read_graphml(outputPath+sc+'_during.graphml')
B = nx.read_graphml('From 2002-03-25 to 2003-03-19_FI0009000681.graphml')
D = nx.read_graphml('From 2002-11-20 to 2003-11-14_FI0009000681.graphml')
#c1,c2 = common_sub_structure_2(B,D)
#####################
B = update_atr(B)
D  = update_atr(D)
#clique = common_sub_structure(B,D)
clique = ['29', '31', '24', '15', '26', '40', '58', '59', '97', '72', '98',
          '93', '102', '82', '62', '67', '92', '108', '10', '74', '73',
          '63', '1', '56', '104', '107', '9', '78', '79'] # just another clique
B = update_subCommon_structure(B,clique)
D = update_subCommon_structure(D,clique)
B = attributes_for_subcommon(B)   
D = attributes_for_subcommon(D)
B = update_link_atr(B)
D = update_link_atr(D)

nodes = B.nodes()
mapping = {}
for node in nodes:
    mapping.update({node:"before_" + node})
BB = nx.relabel_nodes(B, mapping)

both = nx.compose(BB, D)


B_json, D_json = nx.node_link_data(B), nx.node_link_data(D)
B_json['edges'] = B_json.pop('links')
D_json['edges'] = D_json.pop('links')

for i in range(len(D_json['edges'])):
    D_json['edges'][i]['id'] = str(i)
    
for i in range(len(B_json['edges'])):
    B_json['edges'][i]['id'] = str(i)

import json
with open('../public/data/before_new.json','w') as jf1:
    json.dump(B_json, jf1)
with open('../public/data/during_new.json','w') as jf2:
    json.dump(D_json, jf2)

In [58]:
nx.write_graphml(B, "before_with_attrs.graphml")
nx.write_graphml(D, "during_with_attrs.graphml")

nx.write_graphml(both, "both.graphml")